<a href="https://colab.research.google.com/github/syntactic/DeepReinforcementLearning/blob/main/Group9_HW4_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 Homework Review
This task asks you to review two other groups’ homework. The goal includes (1) for you to get a better understanding of contents by reviewing other groups submissions, (2) helping them understand how they could improve with their code (and possibly RL), and (3) help you improve by receiving valuable feedback from other groups. Step-by-step:
1. Coordinate with two other groups for mutual feedback. You may use the forum to achieve this, but we also try to match groups spontaneously at each QnA.
1
2. Take 15-30 min each to review their respective submissions. Write bullet points on your findings (both what your group should learn from their submission, and what the other group should improve)
3. Get together and discuss this feedback with representatives of all three groups in one of either the in-person or digital QnA sessions. Have one of the attending tutors as a ’referee’ for any upcoming discussion and questions, and make sure they write down having refereed your group.
4. Denote the groups and respective tutor in the homework submission form


### 2 DQN
This homework asks you to implement DQN on the [ALE Breakout v5 Atari Game](https://gymnasium.farama.org/environments/atari/breakout/)
* Achieving a reasonable score takes four to twenty four hours on a reasonable computer system with dedicated GPU. plan accordingly and build towards efficiency and throughput (make use of vectorized environments!).
* If you do not have the necessary compute ressources available, you may instead solve the discrete version of lunar lander (also available on gym- nasium!). Notice you can not be awarded an outstanding in this case however!
* Make use of the [Implementing DQN from scratch video](https://www.youtube.com/playlist?list=PLPitqsshnVV8YOGE1r-Sm2zVtuSsGNL_G) series if necessary.
* Make use of a delayed target network, prefilling the ERP and some additional measures to tackle the overestimation bias (e.g. Double DQN)

In [1]:
pip install gymnasium[atari,accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 43.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.2 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=01f0266912c142f8cf99719ea949b6f528c3599566122350e357c9e0e49997dd
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [2]:
import tensorflow as tf
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime

In [3]:
class ExperienceReplayBuffer:

    def __init__(self, max_size: int, environment_name: str, parallel_game_unrolls: int, observation_preprocessing_function: callable, unroll_steps:int):
        self.max_size = max_size
        self.environment_name = environment_name
        self.parallel_game_unrolls = parallel_game_unrolls
        self.unroll_steps = unroll_steps
        self.observation_preprocessing_function = observation_preprocessing_function
        self.num_possible_actions = gym.make(environment_name).action_space.n
        self.envs = gym.vector.make(environment_name, num_envs=self.parallel_game_unrolls)
        self.current_states, _ = self.envs.reset()
        self.data = []

    def fill_with_samples(self, dqn_network, epsilon: float):
        # adds new samples into the ERP

        states_list = []
        actions_list = []
        rewards_list = []
        terminateds_list = []
        next_states_list = []

        
        for i in range(self.unroll_steps):
            actions = self.sample_epsilon_greedy(dqn_network, epsilon) # (PARALLEL_GAME_UNROLLS, )
            # take the action and get s' and r
            next_states, rewards, terminateds, _, _ = self.envs.step(actions)
            # store observation, action, reward, next_observation into ERP container
            #
            states_list.append(self.current_states)
            actions_list.append(actions)
            rewards_list.append(rewards)
            terminateds_list.append(terminateds)
            next_states_list.append(next_states)
            self.current_states = next_states

        def data_generator():
            for states_batch, actions_batch, rewards_batch, terminateds_batch, next_states_batch in \
                zip(states_list, actions_list, rewards_list, terminateds_list, next_states_list):
                for game_idx in range(self.parallel_game_unrolls):
                    state = states_batch[game_idx,:,:,:]
                    action = actions_batch[game_idx]
                    reward = rewards_batch[game_idx]
                    terminated = terminateds_batch[game_idx]
                    next_state = next_states_batch[game_idx,:,:,:]
                    yield(state, action, reward, next_state, terminated)
        
        dataset_tensor_specs = (tf.TensorSpec(shape=(210,160,3), dtype=tf.uint8), 
                                tf.TensorSpec(shape=(), dtype=tf.int32), 
                                tf.TensorSpec(shape=(), dtype=tf.float32), 
                                tf.TensorSpec(shape=(210,160,3), dtype=tf.uint8),
                                tf.TensorSpec(shape=(), dtype=tf.bool))
        new_samples_dataset = tf.data.Dataset.from_generator(data_generator, output_signature=dataset_tensor_specs)
        
        new_samples_dataset = new_samples_dataset.map(lambda state, action, reward, next_state, terminated:(self.observation_preprocessing_function(state), action, reward, self.observation_preprocessing_function(next_state), terminated))
        new_samples_dataset = new_samples_dataset.cache().shuffle(buffer_size=self.unroll_steps * self.parallel_game_unrolls, reshuffle_each_iteration=True)

        for elem in new_samples_dataset:
            continue

        self.data.append(new_samples_dataset)

        datapoints_in_data = len(self.data) * self.parallel_game_unrolls * self.unroll_steps
        if datapoints_in_data > self.max_size:
            self.data.pop(0)


    def create_dataset(self):
        ERP_dataset = tf.data.Dataset.sample_from_datasets(self.data, weights=[1/float(len(self.data)) for _ in self.data], stop_on_empty_dataset = False)
        return ERP_dataset

    def sample_epsilon_greedy(self, dqn_network, epsilon):
        observations = self.observation_preprocessing_function(self.current_states)
        q_values = dqn_network(observations) # tensor float 32 shape(parallel_game_unrolls, num_actions)
        greedy_actions = tf.argmax(q_values, axis=1)
        random_actions = tf.random.uniform(shape=(self.parallel_game_unrolls,), minval=0, maxval=self.num_possible_actions, dtype=tf.int64)
        epsilon_sampling = tf.random.uniform(shape=(self.parallel_game_unrolls,), minval=0, maxval=1, dtype=tf.float32) > epsilon
        actions = tf.where(epsilon_sampling, greedy_actions, random_actions).numpy()
        return actions

def observation_preprocessing_function(observation):
    # preprocess our observation so that it has shape (84, 84) and is between -1 and 1
    observation = tf.image.resize(observation, size=(84,84))
    observation = tf.cast(observation, dtype=tf.float32)/128.0 - 1.0
    return observation

def create_dqn_model(num_actions: int):
    # create intput for function tf model api
    input_layer = tf.keras.Input(shape=(84,84,3), dtype=tf.float32)

    x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', padding='same')(input_layer) # (84, 84, 3)
    x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', padding='same')(input_layer) + x # residual connections
    x = tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', padding='same')(input_layer) + x

    x = tf.keras.layers.MaxPool2D(pool_size=2)(x) # (42, 42, )

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x) # (42, 42, )
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x) + x
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(x) + x

    x = tf.keras.layers.MaxPool2D(pool_size=2)(x) #(21, 21, )

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) #(21, 21, )
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) + x
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) + x 

    x = tf.keras.layers.MaxPool2D(pool_size=2)(x) #(10, 10, )

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) + x
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) + x
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x) + x

    x = tf.keras.layers.GlobalAvgPool2D()(x)

    x = tf.keras.layers.Dense(units=64, activation='relu')(x) + x
    x = tf.keras.layers.Dense(units=num_actions, activation='linear')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=x)

    return model

def train_dqn(train_dqn_network, target_network, dataset, optimizer, gamma: float, num_training_steps: int, batch_size: int=256):
    dataset = dataset.batch(batch_size).prefetch(4)
    @tf.function
    def training_step(q_target, observations, actions):
        with tf.GradientTape() as tape:
            q_predictions_all_actions = train_dqn_network(observations) # shape of q_predictions is (batch_size, num_actions)
            q_predictions = tf.gather(q_predictions_all_actions, actions, batch_dims=1)
            loss = tf.reduce_mean(tf.square(q_predictions - q_target))
        gradients = tape.gradient(loss, train_dqn_network.trainable_variables)
        optimizer.apply_gradients(zip(gradients, train_dqn_network.trainable_variables))
        return loss

    losses = []
    q_values = []
    for i, state_transition in enumerate(dataset):
        state, action, reward, subsequent_state, terminated = state_transition
        # calculate q_target
        #print(subsequent_state.shape)
        q_vals = target_network(subsequent_state)
        q_values.append(q_vals.numpy())
        max_q_values = tf.reduce_max(q_vals, axis=1)
        use_subsequent_state = tf.where(terminated, tf.zeros_like(max_q_values, dtype=tf.float32), tf.ones_like(max_q_values, dtype=tf.float32))
        q_target = reward + (gamma*max_q_values*use_subsequent_state)
        loss = training_step(q_target, observations=state, actions=action)
        losses.append(loss)
        if i >= num_training_steps:
            break
    return np.mean(losses), np.mean(q_values)

def test_q_network(test_dqn_network, environment_name: str, num_parallel_tests: int, gamma: float, preprocessing_function: callable, test_epsilon: float=0.05):
    envs = gym.vector.make(environment_name, num_parallel_tests)
    num_possible_actions = envs.single_action_space.n
    states, _ = envs.reset()
    done = False
    timestep = 0
    # episodes_finished is np vector of shape (num_parallel_tests,), filled with booleans, starting with all False
    episodes_finished = np.zeros(num_parallel_tests, dtype=bool)
    returns = np.zeros(num_parallel_tests)
    test_steps = 0
    while not done:
        states = preprocessing_function(states)
        q_values = test_dqn_network(states)
        greedy_actions = tf.argmax(q_values, axis=1) # tensor of type tf.int64, shape (num_parallel_tests,)
        random_actions = tf.random.uniform(shape=(num_parallel_tests, ), minval=0,
                                           maxval=num_possible_actions, dtype=tf.int64)
        epsilon_sampling = tf.random.uniform(shape=(num_parallel_tests,), minval=0,
                                             maxval=1, dtype=tf.float32) > test_epsilon # tensor of type tf.bool, shape (num_parallel_tests,)
        actions = tf.where(epsilon_sampling, greedy_actions, random_actions).numpy() # tensor of type tf.int64, shape (num_parallel_tests,)
        states, rewards, terminateds, _, _ = envs.step(actions)
        # compute pointwise or between episodes_finished and terminateds
        episodes_finished = np.logical_or(episodes_finished, terminateds)
        returns += ((gamma**timestep)*rewards)*(np.logical_not(episodes_finished).astype(np.float32))
        timestep += 1
        # done if all episodes are finished
        done = np.all(episodes_finished)
        test_steps += 1
        if test_steps % 100 == 0:
           print(f"test_steps: {test_steps} {np.sum(episodes_finished)/num_parallel_tests} {terminateds.shape} {episodes_finished.shape}")
    return np.mean(returns)

def visualize_q_values(results_df, step):
    # create three subplots
    fig, axis = plt.subplots(1, 3)
    # include the row idxs explicitly in the results_df
    results_df['step'] = results_df.index
    # plot the average return
    sns.lineplot(x='step', y='average_return', data=results_df, ax=axis[0])
    # plot the average loss
    sns.lineplot(x='step', y='average_loss', data=results_df, ax=axis[1])
    # plot the average q values
    sns.lineplot(x='step', y='average_q_values', data=results_df, ax=axis[2])
    # save the figure
    # create a timestring from the timestamp
    timestring = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # save the figure
    plt.savefig(f"./{timestring}_results_step{step}.png")
    # close the figure
    plt.close(fig)


def polyak_averaging_weights(source_network, target_network, polyak_averaging_factor: float):
    source_network_weights = source_network.get_weights()
    target_network_weights = target_network.get_weights()
    averaged_weights = []
    for source_weight, target_weight in zip(source_network_weights, target_network_weights):
        fraction_kept_weights = polyak_averaging_factor * target_weight
        fraction_updated_weights = (1-polyak_averaging_factor) * source_weight
        averaged_weight = fraction_kept_weights + fraction_updated_weights
        averaged_weights.append(averaged_weight)
    target_network.set_weights(averaged_weights)

def dqn():
    ENVIRONMENT_NAME = "ALE/Breakout-v5"
    NUMBER_ACTIONS = gym.make(ENVIRONMENT_NAME).action_space.n
    ERP_SIZE = 10000
    PARALLEL_GAME_UNROLLS = 64
    UNROLL_STEPS = 4
    EPSILON = 0.2
    GAMMA = 0.98
    NUM_TRAINING_STEPS_PER_ITER = 4
    NUM_TRAINING_ITERS = 50000
    TEST_EVERY_N_STEPS = 1000
    TEST_NUM_PARALLEL_ENVS = 128
    PREFILL_STEPS = 50
    POLYAK_AVERAGING_FACTOR = 0.99
    erp = ExperienceReplayBuffer(max_size=ERP_SIZE, environment_name=ENVIRONMENT_NAME, 
                                 parallel_game_unrolls=PARALLEL_GAME_UNROLLS, unroll_steps=UNROLL_STEPS,
                                 observation_preprocessing_function=observation_preprocessing_function)
    
    # This is the DQN we train
    dqn_agent = create_dqn_model(num_actions=NUMBER_ACTIONS)
    # This is the target network, used to calculate the q-estimation targets
    target_network = create_dqn_model(num_actions=NUMBER_ACTIONS)
    dqn_agent.summary()
    # test the agent
    dqn_agent(tf.random.uniform(shape=(1, 84, 84, 3)))
    # copy over the weights from the dqn_agent to the target_network via polyak averaging with factor 0.0
    polyak_averaging_weights(dqn_agent, target_network, polyak_averaging_factor=0.0)

    dqn_optimizer = tf.keras.optimizers.Adam()

    return_tracker = []
    dqn_prediction_error = []
    average_q_values = []

    # prefill the replay buffer
    prefill_exploration_epsilon = 1.
    for prefill_step in range(PREFILL_STEPS):
        erp.fill_with_samples(dqn_agent, epsilon=prefill_exploration_epsilon)


    for step in range(NUM_TRAINING_ITERS):
        print(f'Training step: {step}')
        # step 1: put some s, a, r, s', t transitions into the replay buffer
        erp.fill_with_samples(dqn_agent, epsilon=EPSILON)
        dataset = erp.create_dataset()
        # step 2: train on some samples from the replay buffer
        average_loss, average_q_vals = train_dqn(dqn_agent, target_network, dataset, dqn_optimizer, gamma=GAMMA, num_training_steps=NUM_TRAINING_STEPS_PER_ITER)
        # update the target network via polyak averaging
        polyak_averaging_weights(dqn_agent, target_network, polyak_averaging_factor=POLYAK_AVERAGING_FACTOR)

        if step % TEST_EVERY_N_STEPS == 0:
            average_return = test_q_network(dqn_agent, ENVIRONMENT_NAME, num_parallel_tests=TEST_NUM_PARALLEL_ENVS, gamma=GAMMA,
                                            preprocessing_function=observation_preprocessing_function)
            return_tracker.append(average_return)
            dqn_prediction_error.append(average_loss)
            average_q_values.append(average_q_vals)
            # print average returns, average loss, average q values
            print(f"TESTING: Average return: {average_return}, Average loss: {average_loss}, Average q value-estimation: {average_q_vals}")
            # put all result lists into a dataframe by transforming them into a dict first
            results_dict = {'average_return': return_tracker, 'average_loss': dqn_prediction_error, 'average_q_values': average_q_values}
            results_df = pd.DataFrame(results_dict)
            # visualize the results with sns
            visualize_q_values(results_df, step)
            print(results_df)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dqn()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 84, 84, 3)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 84, 84, 16)   448         ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 84, 84, 16)   448         ['input_1[0][0]']                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 84, 84, 16)   448         ['input_1[0][0]']                
                                                                                              